## Load Dataset

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('loan.csv')
df.head() # View the data

## Dataset Info

In [ ]:
df.info()

## Data Cleaning

In [ ]:
# We deal with NULL rows to finish task1
df.drop("Loan_ID", axis=1, inplace=True)
# checking the Missing Values
df.isnull().sum()

In [ ]:
# We choose to drop the NULL rows
new_df = df.dropna()

## Encode

In [ ]:
# We use label encoding
new_df.Gender = new_df.Gender.map({'Male':1,'Female':0})
new_df.Married = new_df.Married.map({'Yes':1,'No':0})
new_df.Dependents = new_df.Dependents.map({'0':0,'1':1,'2':2,'3+':3})
new_df.Education = new_df.Education.map({'Graduate':1,'Not Graduate':0})
new_df.Self_Employed = new_df.Self_Employed.map({'Yes':1,'No':0})
new_df.Property_Area = new_df.Property_Area.map({'Urban':0,'Semiurban':1,'Rural':2})
new_df.Loan_Status = new_df.Loan_Status.map({'Y':1,'N':0})

## Data process

In [ ]:
# Split the dataset by the bootstraping method, since the number
# of the whole dataset is so small
train_set = new_df.sample(n = new_df.shape[0],replace = True,axis = 0)
temp = train_set.drop_duplicates()
test_set = pd.concat([new_df,temp,temp]).drop_duplicates(keep = False)

In [ ]:
# Split X and y
X_train = train_set.iloc[:,0:11]
y_train = train_set['Loan_Status']
X_test = test_set.iloc[:,0:11]
y_test = test_set['Loan_Status']
# Normalization(Min-Max mothod)
X_train_norm = (X_train-X_train.min())/(X_train.max()-X_train.min())
y_train_norm = (y_train-y_train.min())/(y_train.max()-y_train.min())
X_test_norm = (X_test-X_test.min())/(X_test.max()-X_test.min())
y_test_norm = (y_test-y_test.min())/(y_test.max()-y_test.min())

## Train

In [ ]:
from Logistic import LogisticRegression
model = LogisticRegression()
# The train with no regularzation, the learning rate is 0.01
cost_list,params,grads = model.Gradient_descent_train(X_train_norm,y_train_norm,0.01,10000,0,0)

In [ ]:
# Plot the loss curve of training above
import matplotlib.pyplot as plt
epoch = np.linspace(100,100000,1000)
plt.plot(epoch,cost_list)
plt.title('Loss Function')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

## Test and evaluation

In [ ]:
y_train_predict = model.predict_test(X_train_norm,params)
y_test_predict = model.predict_test(X_test_norm,params)

In [ ]:
# calculate the accuracy to evaluate the model
accuracy_score_train = model.accuracy_metric(y_train_norm,y_train_predict)
accuracy_score_test = model.accuracy_metric(y_test_norm,y_test_predict)
print('train accuracy is: ',accuracy_score_train)
print('test accuracy is: ',accuracy_score_test)

## Adjust the parameters

In [ ]:
# l1 regularization with the parameters lamada = 1
cost_list,params,grads = model.Gradient_descent_train(X_train_norm,y_train_norm,0.01,100000,1,1)
y_train_predict = model.predict_test(X_train_norm,params)
y_test_predict = model.predict_test(X_test_norm,params)
accuracy_score_train = model.accuracy_metric(y_train_norm,y_train_predict)
accuracy_score_test = model.accuracy_metric(y_test_norm,y_test_predict)
print('train accuracy with l1 regularizationis: ',accuracy_score_train)
print('test accuracy with l1 regularization is: ',accuracy_score_test)

In [ ]:
# l2 regularization with the parameters lamada = 1
cost_list,params,grads = model.Gradient_descent_train(X_train_norm,y_train_norm,0.01,10000,2,2)
y_train_predict = model.predict_test(X_train_norm,params)
y_test_predict = model.predict_test(X_test_norm,params)
accuracy_score_train = model.accuracy_metric(y_train_norm,y_train_predict)
accuracy_score_test = model.accuracy_metric(y_test_norm,y_test_predict)
print('train accuracy with l2 regularization is: ',accuracy_score_train)
print('test accuracy with l2 regularization is: ',accuracy_score_test)

In [ ]:
# adjusting the learning rate with no regularization
learning_rate = 0.05
cost_list,params,grads = model.Gradient_descent_train(X_train_norm,y_train_norm,learning_rate,10000,0,0)
y_train_predict = model.predict_test(X_train_norm,params)
y_test_predict = model.predict_test(X_test_norm,params)
accuracy_score_train = model.accuracy_metric(y_train_norm,y_train_predict)
accuracy_score_test = model.accuracy_metric(y_test_norm,y_test_predict)
print('train accuracy is: ',accuracy_score_train)
print('test accuracy is: ',accuracy_score_test)

## Optimizing Model Evaluation Methods: 5 times Bootstrapping and get the mean of accuracy


In [ ]:
# initialize
accuracy_train = []
accuracy_test = []

for i in range(5):
    train_set = new_df.sample(n = new_df.shape[0],replace = True,axis = 0)
    temp = train_set.drop_duplicates()
    test_set = pd.concat([new_df,temp,temp]).drop_duplicates(keep = False)
    # Split X and y
    X_train = train_set.iloc[:,0:11]
    y_train = train_set['Loan_Status']
    X_test = test_set.iloc[:,0:11]
    y_test = test_set['Loan_Status']
    # Normalization(Min-Max mothod)
    X_train_norm = (X_train-X_train.min())/(X_train.max()-X_train.min())
    y_train_norm = (y_train-y_train.min())/(y_train.max()-y_train.min())
    X_test_norm = (X_test-X_test.min())/(X_test.max()-X_test.min())
    y_test_norm = (y_test-y_test.min())/(y_test.max()-y_test.min())

    from Logistic import LogisticRegression
    model = LogisticRegression()

    # We can adjust the parameters here
    # Train
    cost_list,params,grads = model.Gradient_descent_train(X_train_norm,y_train_norm,0.01,10000,2,1)
    
    # Test
    y_train_predict = model.predict_test(X_train_norm,params)
    y_test_predict = model.predict_test(X_test_norm,params)
    # Accuracy
    accuracy_train.append(model.accuracy_metric(y_train_norm,y_train_predict))
    accuracy_test.append(model.accuracy_metric(y_test_norm,y_test_predict))

acc_train = sum(accuracy_train)/len(accuracy_train)
acc_test = sum(accuracy_test)/len(accuracy_test)

print('train accuracy is: ',acc_train)
print('test accuracy is: ',acc_test)